<a href="https://colab.research.google.com/github/shainaraza/wikinews-scrape/blob/master/downloading_and_parsing_wikinews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*   Finding and retrieve online data
*   Parsing XML using a SAX parser
*  Parsing Mediawiki content using mwparserfromhell
*  Running operations in parallel using multiprocessing and multithreading



In [0]:
import requests

# Parsing HTML
from bs4 import BeautifulSoup

# File system management
import os

# Searching for Wikipedia Dumps


In [0]:
#base_url = 'https://dumps.wikimedia.org/enwiki/'
base_url = 'https://dumps.wikimedia.org/enwikinews/'
index = requests.get(base_url).text
soup_index = BeautifulSoup(index, 'html.parser')

# Find the links that are dates of dumps
dumps = [a['href'] for a in soup_index.find_all('a') if 
         a.has_attr('href')]
dumps

#for now i am taking april 2020, later i will take a few from all these three years for tempioral dynamics insh a ALLAH 

['../',
 '20191101/',
 '20191120/',
 '20191201/',
 '20191220/',
 '20200101/',
 '20200120/',
 '20200201/',
 '20200220/',
 '20200301/',
 '20200401/',
 'latest/']

In [0]:
dump_url = base_url + '20200401/'

# Retrieve the html
dump_html = requests.get(dump_url).text
dump_html[:10]

'<!DOCTYPE '

In [0]:
# Convert to a soup
soup_dump = BeautifulSoup(dump_html, 'html.parser')

# Find li elements with the class file
soup_dump.find_all('li', {'class': 'file'}) #[:4]

In [0]:
files = []

# Search through all files
for file in soup_dump.find_all('li', {'class': 'file'}):
    text = file.text
    # Select the relevant files
    if 'pages-articles' in text:
        files.append((text.split()[0], text.split()[1:]))
        
files[:5]

[('enwikinews-20200401-pages-articles-multistream.xml.bz2', ['45.7', 'MB']),
 ('enwikinews-20200401-pages-articles-multistream-index.txt.bz2',
  ['1.2', 'MB']),
 ('enwikinews-20200401-pages-articles.xml.bz2', ['39.9', 'MB'])]

In [0]:
files_to_download = [file[0] for file in files if '.xml' in file[0]]
files_to_download[-5:]

['enwikinews-20200401-pages-articles-multistream.xml.bz2',
 'enwikinews-20200401-pages-articles.xml.bz2']

Download Wikipedia Data


In [0]:
import sys

keras_home = '/content/drive/My Drive/Shaina- DL NRS/data preprocessing/wikinews/'

In [0]:
from urllib.request import urlretrieve
import os,sys
from zipfile import ZipFile
from pathlib import Path
from keras.utils import get_file

data_paths = []
file_info = []

# Iterate through each file
for file in files_to_download[:5]:
     path = keras_home + file
   

     if not os.path.exists( keras_home + file):
        print('Downloading')
        # If not, download the file
        data_paths.append(get_file(keras_home +file, dump_url+file))
       

        # Find the file size in MB
        file_size = os.stat(path).st_size / 1e6
        file_info.append((file, file_size))

     
    # If the file is already downloaded find some information
     else:
        data_paths.append(path)
        # Find the file size in MB
        file_size = os.stat(path).st_size / 1e6
        file_info.append((file, file_size))


In [0]:
print(f'There are {len(file_info)} partitions.')


There are 2 partitions.


In [0]:
print(f"The total size of files on disk is {file_df['size (MB)'].sum() / 1e3} GB")


The total size of files on disk is 0.041848355 GB


In [0]:
import bz2
import subprocess

data_path = data_paths[1]
data_path

'/content/drive/My Drive/Shaina- DL NRS/data preprocessing/wikinews/enwikinews-20200401-pages-articles.xml.bz2'

In [0]:
sorted(file_info, key = lambda x: x[1], reverse = False)[:5]


[('enwikinews-20200401-pages-articles.xml.bz2', 41.848355),
 ('enwikinews-20200401-pages-articles-multistream.xml.bz2', 47.892707)]

In [0]:
#The subprocess + bzcat approach is nearly twice as fast. Let's run this again and see what kind of data we have.
lines = []


for i, line in enumerate(subprocess.Popen(['bzcat'], 
                         stdin = open(data_path), 
                         stdout = subprocess.PIPE).stdout):
    lines.append(line)
lines


In [0]:

lines[-160:-116]


**Parsing Approach
**

> In order to get useful information from this data, we have to parse it on two levels.
1.  Extract the titles and article text from the XML
2.   Extract relevant information from the article text


Parsing XML using XML, SAX


In [0]:
import xml.sax

class WikiXmlHandler(xml.sax.handler.ContentHandler):
    """Content handler for Wiki XML data using SAX"""
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._current_tag = None
        self._pages = []

    def characters(self, content):
        """Characters between opening and closing tags"""
        if self._current_tag:
            self._buffer.append(content)

    def startElement(self, name, attrs):
        """Opening tag of element"""
        if name in ('title', 'text', 'timestamp'):
            self._current_tag = name
            self._buffer = []

    def endElement(self, name):
        """Closing tag of element"""
        if name == self._current_tag:
            self._values[name] = ' '.join(self._buffer)
        if name == 'page':
            self._pages.append((self._values['title'], self._values['text']))
      

In [0]:
# Object for handling xml
handler = WikiXmlHandler()

# Parsing object
parser = xml.sax.make_parser()
parser.setContentHandler(handler)

for i, line in enumerate(subprocess.Popen(['bzcat'], 
                         stdin = open(data_path), 
                         stdout = subprocess.PIPE).stdout):
    parser.feed(line)
    
 
print([x[0] for x in handler._pages])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
handler._pages[100]

('Yasser Arafat funeral representing Brazil comitive has back today',
 '#REDIRECT [[Brazilian delegation returns from Arafat funeral]] \n \n [[Category:Protected mainspace redirects]]')

In [0]:
!pip install mwparserfromhell 

In [0]:
def process_article(title, text, timestamp):
   wikicode = mwparserfromhell.parse(text)
   wikilinks = [x.title.strip_code().strip() for x in wikicode.filter_wikilinks()]

        # Extract external links
   exlinks = [x.url.strip_code().strip() for x in wikicode.filter_external_links()]

         # Find approximate length of article
   text_length = len(wikicode.strip_code().strip())

   return (title,  wikilinks, exlinks, timestamp, text_length)

In [0]:
class WikiXmlHandler(xml.sax.handler.ContentHandler):
    """Parse through XML data using SAX"""
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._current_tag = None
        self._news = []
        self._article_count = 0
        self._non_matches = []

    def characters(self, content):
        """Characters between opening and closing tags"""
        if self._current_tag:
            self._buffer.append(content)

    def startElement(self, name, attrs):
        """Opening tag of element"""
        if name in ('title', 'text', 'timestamp'):
            self._current_tag = name
            self._buffer = []

    def endElement(self, name):
        """Closing tag of element"""
        if name == self._current_tag:
            self._values[name] = ' '.join(self._buffer)

        if name == 'page':
            self._article_count += 1
            # Search through the page to see if the page is a newsitem
            news_item = process_article(**self._values )
            # Append to the list of news
            if news_item:
                self._news.append(news_item)

In [0]:
# Object for handling xml
handler = WikiXmlHandler()

# Parsing object
parser = xml.sax.make_parser()
parser.setContentHandler(handler)

for i, line in enumerate(subprocess.Popen(['bzcat'], 
                         stdin = open(data_path), 
                         stdout = subprocess.PIPE).stdout):
    parser.feed(line)
 
        
print(f'Searched through {handler._article_count} .')

Searched through 99352 .


In [0]:
handler._news[106]


('Colin Powell Resigns as Secretary of State, Rice Likely Successor',
 ['Colin Powell Resigns as U.S. Secretary of State, Rice Likely Successor',
  'Category:Protected mainspace redirects'],
 [],
 '2012-08-07T21:27:56Z',
 122)

In [0]:
# Uncompress the file if not already uncompressed
if not os.path.exists('/content/drive/My Drive/Shaina- DL NRS/data preprocessing/wikinews/articles11.xml'):
    subprocess.call(["bzcat '/content/drive/My Drive/Shaina- DL NRS/data preprocessing/wikinews/enwikinews-20200401-pages-articles.xml.bz2' >> '/content/drive/My Drive/Shaina- DL NRS/data preprocessing/wikinews/articles11.xml'"],
                    shell = True)
else:
    print('Already uncompressed')

Already uncompressed


In [0]:
!wc '/content/drive/My Drive/Shaina- DL NRS/data preprocessing/wikinews/articles11.xml'


  3704634  20903437 204994714 /content/drive/My Drive/Shaina- DL NRS/data preprocessing/wikinews/articles11.xml


In [0]:

handler = WikiXmlHandler()

# Parsing object
parser = xml.sax.make_parser()
parser.setContentHandler(handler)

# Parse the entire file
for i, line in enumerate(subprocess.Popen(['bzcat'], 
                         stdin = open(data_path), 
                         stdout = subprocess.PIPE).stdout):
    parser.feed(line)
news = handler._news

print(f'\nSearched through {handler._article_count} articles.')
print(f'\nFound {len(news)} newsbooks .')


Searched through 99352 articles.

Found 99352 newsbooks .


In [0]:

import json

# Save list of books
with open('/content/drive/My Drive/Shaina- DL NRS/data preprocessing/wikinews/p15_books.ndjson', 'wt') as fout:
    for l in books:
        fout.write(json.dumps(l) + '\n')

In [0]:
news_in = []

# Read in list of news
with open('/content/drive/My Drive/Shaina- DL NRS/data preprocessing/wikinews/p15_books.ndjson', 'rt') as fin:
    for l in fin.readlines():
       news_in.append(json.loads(l))

In [0]:
news_in


In [0]:

# Save list of books
with open('/content/drive/My Drive/Shaina- DL NRS/data preprocessing/wikinews/p15_books.ndjson', 'wt') as fout:
    for l in news:
        fout.write(json.dumps(l) + '\n')

In [0]:
news_in[100]

['Yasser Arafat funeral representing Brazil comitive has back today',
 ['Brazilian delegation returns from Arafat funeral',
  'Category:Protected mainspace redirects'],
 [],
 '2012-08-07T21:16:35Z',
 100]

In [0]:
import gc
import json

def find_news(data_path, limit = None, save = True):
    """Find all the book articles from a compressed wikipedia XML dump.
       `limit` is an optional argument to only return a set number of books.
        If save, books are saved to partition directory based on file name"""

    # Object for handling xml
    handler = WikiXmlHandler()

    # Parsing object
    parser = xml.sax.make_parser()
    parser.setContentHandler(handler)

    # Iterate through compressed file
    for i, line in enumerate(subprocess.Popen(['bzcat'], 
                             stdin = open(data_path), 
                             stdout = subprocess.PIPE).stdout):
        try:
            parser.feed(line)
        except StopIteration:
            break
            
        # Optional limit
        if limit is not None and len(handler._news) >= limit:
            return handler._news
    
    if save:
        partition_dir = '/content/drive/My Drive/Shaina- DL NRS/data preprocessing/wikinews/partitions/'
        # Create file name based on partition name
        p_str = data_path.split('-')[-1].split('.')[-2]
        out_dir = partition_dir + f'{p_str}.ndjson'

        # Open the file
        with open(out_dir, 'w') as fout:
            # Write as json
            for news_item in handler._news:
                fout.write(json.dumps(news_item) + '\n')
        
        print(f'{len(os.listdir(partition_dir))} files processed.', end = '\r')

    # Memory management
    del handler
    del parser
    gc.collect()
    return None

In [0]:
partitions = [keras_home + file for file in os.listdir(keras_home) if '-pages-articles.xml' in file]
len(partitions), partitions[-1]

(1,
 '/content/drive/My Drive/Shaina- DL NRS/data preprocessing/wikinews/enwikinews-20200401-pages-articles.xml.bz2')

In [0]:
from multiprocessing import Pool 
import tqdm 

# List of lists to single list
from itertools import chain

# Sending keyword arguments in map
from functools import partial

In [0]:
os.cpu_count()


2

In [0]:
# Create a pool of workers to execute processes
pool = Pool(processes = 16)

start = timer()

# Map (service, tasks), applies function to each partition
results = pool.map(find_news, partitions)

pool.close()
pool.join()

end = timer()
print(f'{end - start} seconds elapsed.')

117.88727112699962 seconds elapsed.


In [0]:
def read_data(file_path):
    """Read in json data from `file_path`"""
    
    data = []
    
    # Open the file and load in json
    with open(file_path, 'r') as fin:
        for l in fin.readlines():
            data.append(json.loads(l))
            
    return data

In [0]:
from multiprocessing.dummy import Pool as Threadpool
from itertools import chain

start = timer()

# List of files to read in
saved_files = ['/content/drive/My Drive/Shaina- DL NRS/data preprocessing/wikinews/partitions/' + x for x in os.listdir('/content/drive/My Drive/Shaina- DL NRS/data preprocessing/wikinews/partitions/')]

# Create a threadpool for reading in files
threadpool = Threadpool(processes = 10)

# Read in the files as a list of lists
results = threadpool.map(read_data, saved_files)

# Flatten the list of lists to a single list
news_list = list(chain(*results))

end = timer()

print(f'Found {len(news_list)} news in {round(end - start)} seconds.')

Found 99352 news in 2 seconds.


In [0]:
if not os.path.exists(os.getcwd() + '/content/drive/My Drive/Shaina- DL NRS/data preprocessing/wikinews/found_news_filtered.ndjson'):
    with open('/content/drive/My Drive/Shaina- DL NRS/data preprocessing/wikinews/found_news_filtered.ndjson', 'wt') as fout:
        for news_item in news_list:
             fout.write(json.dumps(news_item) + '\n')
    print('news saved.')
else:
    print('Files already saved.')

news saved.
